In [ ]:
pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 39.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=901116d53afed515d727fa01c99ee1c3a80c576b47ef560edb65db170e82743f
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
pip install langid


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 35.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=8309af83ba9e95422f984b275d57f27c11dcfb133b1dd71cf5d9d9e45bb21059
  Stored in directory: /root/.cache/pip/wheels/32/6a/b6/b7eb43a6ad55b139c15c5daa29f3707659cfa6944d3c696f5b
Successfully built langid


In [ ]:
pip install nltk


In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)


In [ ]:
!pip install hmmlearn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 8.8 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore", message="The default value of `n_init` will change from 10 to 'auto' in 1.4")


In [ ]:

import sys
import warnings
import nltk
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from hmmlearn import hmm

# Suppress warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=UserWarning)

# Load the dataset and select only the first 3000 rows
languages = pd.read_csv('/content/Language Detection.csv').head(2280)

# Split the data into training and testing sets
train_data, test_data = train_test_split(languages, test_size=0.2)

# Function to extract N-gram features
def extract_ngrams(text, n):
    tokens = word_tokenize(text)
    return [' '.join(gram) for gram in ngrams(tokens, n)]

# Extract N-gram features for the training data
X_train = train_data['Text']
y_train = train_data['Language']

# Use CountVectorizer to convert text data into a matrix of token counts
vectorizer = CountVectorizer()
X_train_count = vectorizer.fit_transform(X_train)

# Train the Naive Bayes classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_count, y_train)

# Train the Decision Tree classifier
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train_count, y_train)

# Train the HMM model for each language
hmm_models = {}
for language in languages['Language'].unique():
    language_data = languages[languages['Language'] == language]
    hmm_model = hmm.GaussianHMM(n_components=2)  # Use GaussianHMM for faster training
    hmm_model.fit(vectorizer.transform(language_data['Text']).toarray())
    hmm_models[language] = hmm_model

# Function to predict languages using HMM and Naive Bayes
def predict_languages(text):
    text_count = vectorizer.transform([text])
    nb_prediction = nb_classifier.predict(text_count)
    hmm_predictions = {}
    for language, hmm_model in hmm_models.items():
        hmm_predictions[language] = hmm_model.score(text_count.toarray())
    return nb_prediction, hmm_predictions

# Evaluate the model on the testing data
test_texts = test_data['Text']
test_languages = test_data['Language']
predicted_languages = []
for text in test_texts:
    nb_prediction, hmm_predictions = predict_languages(text)
    predicted_language = max(hmm_predictions, key=hmm_predictions.get)
    predicted_languages.append(predicted_language)

accuracy = accuracy_score(test_languages, predicted_languages)
print('Accuracy:', accuracy)



Accuracy: 0.8793859649122807


In [ ]:
import nltk
nltk.download('punkt')


In [ ]:
import nltk
nltk.data.path.append('/root/nltk_data')


In [ ]:
!pip uninstall -y nltk
!pip install nltk
import nltk
nltk.download('punkt')


In [ ]:
import nltk.data
print(nltk.data.find('tokenizers/punkt'))


/root/nltk_data/tokenizers/punkt


In [ ]:
import nltk
nltk.data.path.append('/root/nltk_data')


In [ ]:
import nltk
nltk.download('all')


In [ ]:
import nltk
import numpy as np
from nltk.tokenize import word_tokenize

# Download tokenizer
nltk.download('punkt')

def predict_sentence_languages(text, hmm_threshold=-1000):
    words = word_tokenize(text)
    detected_languages = set()

    for word in words:
        # Ensure the word exists in the vectorizer vocabulary
        if word not in vectorizer.get_feature_names_out():
            continue  # Skip unseen words

        word_count = vectorizer.transform([word])
        nb_prediction = nb_classifier.predict(word_count)[0]
        hmm_predictions = {}

        # Get HMM predictions
        for language, hmm_model in hmm_models.items():
            try:
                score = hmm_model.score(np.array(word_count.toarray()))  # Ensure proper input format
                if score > hmm_threshold:
                    hmm_predictions[language] = score
            except:
                continue  # Skip if the HMM model fails

        detected_languages.add(nb_prediction)

        if hmm_predictions:
            hmm_prediction = max(hmm_predictions, key=hmm_predictions.get)
            detected_languages.add(hmm_prediction)

    return list(detected_languages)

# Example input
text = input("Enter a sentence: ")
predicted_languages = predict_sentence_languages(text)

print("Detected Languages:", predicted_languages)


malyalam, tamil, kannada, hindi, kokborok, english, punjabi.

In [ ]:

def predict_word_languages(text, hmm_threshold=-1000):  # Set a reasonable score threshold
    words = word_tokenize(text)
    word_languages = {}  # Dictionary to store each word and its predicted language

    for word in words:
        word_count = vectorizer.transform([word])
        nb_prediction = nb_classifier.predict(word_count)[0]
        hmm_predictions = {}

        # Get HMM predictions for the word
        for language, hmm_model in hmm_models.items():
            score = hmm_model.score(word_count.toarray())
            if score > hmm_threshold:  # Only consider languages with scores above the threshold
                hmm_predictions[language] = score

        # Choose the highest scoring HMM prediction if any
        if hmm_predictions:
            hmm_prediction = max(hmm_predictions, key=hmm_predictions.get)
        else:
            hmm_prediction = nb_prediction  # Fallback to Naive Bayes if no HMM prediction

        # Store the word and its predicted language
        word_languages[word] = hmm_prediction

    # Return the dictionary of words and their detected languages
    return word_languages

# Example input text
text = input()
predicted_word_languages = predict_word_languages(text)

# Print the detected language for each word in the sentence
for word, language in predicted_word_languages.items():
    print(f"{word} - {language}")


Unkal peyar enna what is your name aapka name kya hai
Unkal - Malayalam
peyar - Malayalam
enna - Malayalam
what - English
is - English
your - English
name - English
aapka - Malayalam
kya - Malayalam
hai - Malayalam


In [ ]:
import nltk.data
print(nltk.data.find('tokenizers/punkt'))


In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# Ensure necessary downloads
nltk.download('punkt')

def predict_word_languages(text, hmm_threshold=25000, score_margin=2000):
    words = word_tokenize(text)
    word_languages = {}

    for word in words:
        if word in [",", ".", "!", "?", "'", '"']:
            word_languages[word] = 'English'
            continue

        word_count = vectorizer.transform([word])  # Ensure vectorizer is defined
        nb_prediction = nb_classifier.predict(word_count)[0]  # Ensure classifier is defined
        hmm_predictions = {}

        for language, hmm_model in hmm_models.items():  # Ensure hmm_models is defined
            score = hmm_model.score(word_count.toarray())
            print(f"Word: {word}, HMM {language} score: {score}")  # Debugging output
            if score > hmm_threshold:
                hmm_predictions[language] = score

        if hmm_predictions:
            hmm_prediction = max(hmm_predictions, key=hmm_predictions.get)
            hmm_max_score = hmm_predictions[hmm_prediction]

            if hmm_max_score - hmm_predictions.get(nb_prediction, 0) < score_margin:
                final_prediction = nb_prediction
            else:
                final_prediction = hmm_prediction
        else:
            final_prediction = nb_prediction

        word_languages[word] = final_prediction
        print(f"Word: {word}, Naive Bayes prediction: {nb_prediction}, HMM prediction: {final_prediction}")

    return word_languages

# Example input text
text = input("Enter a sentence: ")
predicted_word_languages = predict_word_languages(text)

for word, language in predicted_word_languages.items():
    print(f"{word} - {language}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Enter a sentence: hello aap kese ho
Word: hello, HMM English score: 23506.48267031727
Word: hello, HMM Malayalam score: 1318.1061427863424
Word: hello, HMM Hindi score: 10669.055559417313
Word: hello, HMM Tamil score: 15614.105295650636
Word: hello, Naive Bayes prediction: English, HMM prediction: English
Word: aap, HMM English score: 23679.403166640797
Word: aap, HMM Malayalam score: 28868.10614278634
Word: aap, HMM Hindi score: 10819.055559417313
Word: aap, HMM Tamil score: 26814.105295650636
Word: aap, Naive Bayes prediction: English, HMM prediction: Malayalam
Word: kese, HMM English score: 23679.403166640797
Word: kese, HMM Malayalam score: 28868.10614278634
Word: kese, HMM Hindi score: 10819.055559417313
Word: kese, HMM Tamil score: 26814.105295650636
Word: kese, Naive Bayes prediction: English, HMM prediction: Malayalam
Word: ho, HMM English score: 23679.403166640797
Word: ho, HMM Malayalam score: 28868.10614278634
Word: ho, HMM Hindi score: 10819.055559417313
Word: ho, HMM Tamil

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict languages using Naive Bayes for the test data
nb_predicted_languages = nb_classifier.predict(vectorizer.transform(test_texts))

# Calculate metrics for Naive Bayes
nb_accuracy = accuracy_score(test_languages, nb_predicted_languages)
nb_precision = precision_score(test_languages, nb_predicted_languages, average='weighted')
nb_recall = recall_score(test_languages, nb_predicted_languages, average='weighted')
nb_f1 = f1_score(test_languages, nb_predicted_languages, average='weighted')

print("Naive Bayes Classifier Metrics:")
print(f"Accuracy: {nb_accuracy}")
print(f"Precision: {nb_precision}")
print(f"Recall: {nb_recall}")
print(f"F1 Score: {nb_f1}")


Naive Bayes Classifier Metrics:
Accuracy: 0.9912280701754386
Precision: 0.9912439039837863
Recall: 0.9912280701754386
F1 Score: 0.9911962085484335


In [ ]:
# Predict languages using Decision Tree for the test data
dt_predicted_languages = dt_classifier.predict(vectorizer.transform(test_texts))

# Calculate metrics for Decision Tree
dt_accuracy = accuracy_score(test_languages, dt_predicted_languages)
dt_precision = precision_score(test_languages, dt_predicted_languages, average='weighted')
dt_recall = recall_score(test_languages, dt_predicted_languages, average='weighted')
dt_f1 = f1_score(test_languages, dt_predicted_languages, average='weighted')

print("\nDecision Tree Classifier Metrics:")
print(f"Accuracy: {dt_accuracy}")
print(f"Precision: {dt_precision}")
print(f"Recall: {dt_recall}")
print(f"F1 Score: {dt_f1}")



Decision Tree Classifier Metrics:
Accuracy: 0.9649122807017544
Precision: 0.9698628510902517
Recall: 0.9649122807017544
F1 Score: 0.9651129801614411


In [ ]:
# Initialize list to store HMM predictions
hmm_predicted_languages = []

# Loop through test texts to predict using HMM
for text in test_texts:
    _, hmm_predictions = predict_languages(text)
    predicted_language = max(hmm_predictions, key=hmm_predictions.get)  # The language with the highest score
    hmm_predicted_languages.append(predicted_language)

# Calculate metrics for HMM
hmm_accuracy = accuracy_score(test_languages, hmm_predicted_languages)
hmm_precision = precision_score(test_languages, hmm_predicted_languages, average='weighted')
hmm_recall = recall_score(test_languages, hmm_predicted_languages, average='weighted')
hmm_f1 = f1_score(test_languages, hmm_predicted_languages, average='weighted')

print("\nHidden Markov Model Metrics:")
print(f"Accuracy: {hmm_accuracy}")
print(f"Precision: {hmm_precision}")
print(f"Recall: {hmm_recall}")
print(f"F1 Score: {hmm_f1}")



Hidden Markov Model Metrics:
Accuracy: 0.8925438596491229
Precision: 0.9475015463890843
Recall: 0.8925438596491229
F1 Score: 0.9136341899717371
